Case Mr.Health - Cientista de Dados - DataLakers

Israel Segalin, 04/05/2024

In [ ]:
#Importando as bibliotecas necessárias
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
#Lendo as planilhas de dados para Análise Exploratória
pedidos = pd.read_excel('datasets/pedidos.xlsx')
detalhesPedidos = pd.read_excel('datasets/detalhespedido.xlsx')
itens = pd.read_excel('itens.xlsx')

#Exibe as informações de cada dataset para análise
itens.info()
print('\n')
pedidos.info()
print('\n')
detalhesPedidos.info()


Fazendo uma análise breve nas infos de cada planilha, percebemos que a "itens" cataloga os produtos oferecidos pelo estabelecimento e em seguida seu valor p/unidade (embora esteja incompleto e precisemos nomear as colunas).

Também percebemos que a "detalhespedido" cita os detalhes dos pedidos listados justamente na planilha "pedidos", podendo ser associados pela ID do pedido.

Porém, percebemos que a "detalhespedido" separa o pedido por tipos de produto, mantendo a id, mas em linhas diferentes, causando uma discrepância na quantidade final de linhas em cada tabela, já que a "pedidos", considera apenas o pedido como um todo (considerando todos diferentes produtos).

Para análise posterior, precisaremos preencher a coluna VALOR_TOTAL, que está vazia (NaN), então recriaremos uma única tabela contendo as informações juntas de todas as outras tabelas.

In [ ]:
itens = itens.rename(columns={"Unnamed: 0": "ID_ITEM", 0:"PREÇO_UNIDADE"}) #Corrigindo colunas dos itens
itens.head()

In [ ]:
#Concatenando os datasets de pedidos e detalhes dos pedidos
df = pedidos.merge(detalhesPedidos, on='ID_PEDIDO', how='left')
df = df[['ID_PEDIDO', 'DATA', 'VALOR_TOTAL', 'ID_ITEM', 'QUANTIDADE']] #Deixando apenas as colunas necessárias
df.info()
#Aqui juntamos todos os pedidos com os detalhes, tendo toda divisão de itens mas com as datas da tabela "pedidos"

In [ ]:
#Acima percebemos que a DATA está no tipo OBJECT (String), então irei converte-la para o datetime
df['DATA'] = pd.to_datetime(df['DATA'])

df = pd.merge(df, itens, on="ID_ITEM") #Concatenando os pedidos com os preços_unidade, baseado na ID do item

#Calculando o VALOR_TOTAL para cada linha, multiplicando a quantidade pelo preço_unidade
for index, row in df.iterrows():
    df.at[index, "VALOR_TOTAL"] = (df.at[index, "QUANTIDADE"]) * df.at[index, "PREÇO_UNIDADE"]

df.head(5)

In [ ]:
#Iniciando a análise exploratória em busca de problemas nos dados.

#Verificando se existem valores duplicados
#Aqui abro ponto de dúvida. Neste caso, precisariam ser discutidas as regras de negócio, pois em alguns pedidos, a linha se repete pois se trata do mesmo
#item e mesma quantidade, porém, nada impediria o usuário/cliente de adicionar o mesmo produto no mesmo pedido em duas vezes distintas (começo e fim)
#Mas por consideração ética de que quando o mesmo produto é adicionado no pedido, apenas aumenta a quantidade, removerei os duplicados.
linhas_duplicadas = df[df.duplicated(keep=False)] #Variável para visualizar os valores duplicados

if df.duplicated().any():
    df = df.drop_duplicates()

#Conhecendo os dados
print(df.shape,"\n")
print(df.head(),"\n")
print(df.info(),"\n")

In [ ]:
#A visualização acima já é mais do que suficiente para sabermos que não existem valores nulos, mas executaremos outra análise por segurança.
df.isna().any()

In [ ]:
#Descrição dos valores das colunas para procurar possíveis inconsistências
print(df.describe())

Agora que garanti a qualidade dos dados e o dataset está pronto, posso começar a realizar uma análise descritiva para buscar entender padrões, correlações, comportamentos, e outras características.

In [ ]:
#Observando qual foi o item mais pedido no intervalo de tempo
item_mais_vendido = df.groupby(df['ID_ITEM']).agg({'QUANTIDADE': 'sum'}).reset_index()
plt.bar(item_mais_vendido['ID_ITEM'], item_mais_vendido['QUANTIDADE'], color='skyblue')
item_mais_vendido.head()


In [ ]:
#Observando o mês com mais vendas
vendas_mensais = df.set_index('DATA', inplace=False).resample('ME').sum()
vendas_mensais.reset_index(inplace=True)
vendas_mensais['DATA'] = vendas_mensais['DATA'].dt.strftime('%m/%Y')
plt.plot(vendas_mensais['DATA'], vendas_mensais['VALOR_TOTAL'])
vendas_mensais[['DATA', 'VALOR_TOTAL']].head()

In [ ]:
#Observando vendas diárias
vendas_diarias = df.set_index('DATA', inplace=False).resample('D').sum()
vendas_diarias.reset_index(inplace=True)
vendas_diarias['DATA'] = vendas_diarias['DATA'].dt.strftime('%d/%m')
plt.plot(vendas_diarias['DATA'], vendas_diarias['VALOR_TOTAL'])
vendas_diarias = vendas_diarias.sort_values(by=['VALOR_TOTAL'], ascending=False)
vendas_diarias[['DATA', 'VALOR_TOTAL']].head(10)

Com as observações feitas acima, conseguimos tirar de cara algumas conclusões:

O mês (de acordo com os dados fornecidos) com maior número de vendas e valor arrecadado é Julho. O mês de Setembro tem um valor de apenas 140 reais arrecadados, mas isso se deve ao fato de estarmos considerando apenas o primeiro dia de Setembro.

O produto mais vendido é o item D, com uma diferença considerável para os outros produtos, sendo que a média de quantidade vendida dos outros produtos é 213, com um desvio padrão muito pequeno, enquanto o produto D teve 249 unidades vendidas!

Por fim, na análise do gráfico de vendas diárias, percebemos que o faturamento da rede depende fortemente de picos, sendo que existem dias que não existe nenhum pedido! Após uma análise mais aprofundada, percebi que os dias com maior faturamento são sempre no período de SEXTA-FEIRA até DOMINGO.
Colocando os 10 dias com maior número de vendas em ordem decrescente, percebemos que TODOS os 10 pertencem a este período de dias da semana!

Agora que temos nossas observações concluidas, irei implementar um modelo preditivo para ajudar a calcular a demanda do negócio.
Devido a grande maioria das colunas não servirem para features (por não terem relação direta com a quantidade ou por ainda não serem preenchidas), usarei uma regressão linear, que utilizará as datas cadastradas para previsão das datas futuras. Vale ressaltar que o dataset fornecido é relativamente pequeno, então a precisão do modelo pode ser aumentada junto a implementação de mais dados. Dias já cadastrados também terão maior precisão do que dias não registrados no banco.

In [ ]:
#Agrupando datas e quantidades
diario_data = df.groupby('DATA').agg({'QUANTIDADE': 'sum'}).reset_index()

# Criar variáveis de features (Ano, Mês e Dia)
diario_data['Ano'] = diario_data['DATA'].dt.year
diario_data['Mes'] = diario_data['DATA'].dt.month
diario_data['Dia_do_Ano'] = diario_data['DATA'].dt.dayofyear

# Dividir em conjunto de treinamento e teste
X = diario_data[['Ano', 'Mes', 'Dia_do_Ano']]
y = diario_data['QUANTIDADE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo de regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Imprimir os coeficientes
print("Coeficientes: ", model.coef_)
print("Erro Médio Quadratico: ", metrics.mean_squared_error(y_test, y_pred))
print("Erro Médio Absoluto: ", metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
#Aqui simulei como valor de entrada um dia que já havia sido cadastrado no banco, no qual a quantia consumida foi de 10 unidades
#O resultado da previsão foi de 12 unidades
novo_dado = pd.DataFrame({
    'Ano': [2021],
    'Mes': [6],
    'Dia_do_Ano': [155]
})

previsao = model.predict(novo_dado)
print("Previsão para o dado específico: ", previsao[0])

In [ ]:
#Aqui exibo um gráfico dos dados de testes e os dados gerados pelas predições
plt.figure(figsize=(10, 6))
plt.scatter(X_test['Dia_do_Ano'], y_test, color='green', label='Dados de Teste')
plt.scatter(X_test['Dia_do_Ano'], y_pred, color='red', label='Previsões')
plt.xlabel('Dia do Ano')
plt.ylabel('Quantidade')
plt.title('Previsões vs Dados Reais - Demanda Diária')
plt.legend()
plt.show()

*Avaliação do modelo e Recomendações*

Após todo tratamento dos dados, análise exploratória, análise descritiva, treinamento dos dados e avaliação dos mesmos, tiramos algumas conclusões e insights para recomendar para a Mr.Health.

1. Os dados são passíveis de tratamento, porém, uma mudança na forma da captação para melhor cuidado e organização são sugestões;

2. A quantidade de dados é considerada pequena para um modelo de predição. É valido informar ao cliente leigo na área que quanto maior a quantidade de dados captados, maior a precisão do modelo (Predição mais próxima do número real de demanda);

3. O item D é o produto mais vendido, com boa margem para os outros, então um investimento no marketing deste produto pode alavancar as vendas totais;

4. Dos poucos meses registrados, o mês de Julho foi o com mais vendas, então uma preparação para promoções ou coisas do tipo, são válidas para este mês;

5. Com a análise de vendas diárias, percebemos que a avassaladora maioria dos pedidos são feitos no fim de semana (SEX-SAB-DOM), e pelo gráfico identificamos que a loja depende muito destes picos, então atitudes para garantia de estoque nos finais de semana são consideráveis, além também da empresa considerar o por que de não gerar tantas vendas durante a semana e, tomar medidas cabíveis e possíveis.

6. O estoque necessário para a semana não é tão alto, mas para os finais de semana sim, então dependendo da frequência do fornecimento de estoque, pode ser necessário cargas maiores pensando no futuro próximo (Ex: Caso o fornecimento seja feito semanalmente, escolher um dia como quinta ou sexta feira para não correr riscos de faltar no pico e, caso necessário durante a semana, pequenas quantidades de suprimentos podem ser adquiridas a parte).